In [17]:
import pandas as pd
import numpy as np

In [2]:
path = "../data/route25_potential_and_real_stops.csv"

In [3]:
r25 = pd.read_csv(path)

In [58]:
r25.head(20)

,UniqueNum,Distance from initial stop (feet),SqFtBuffere,Est_TotPop,Est_TotMinority,Est_TotPov,Est_TotLEP,Est_TotPop_Density,Transfer,CorrespondingStopID
0,783,0,4.034183e+06,1464,1254,697,37,7456.09,Yes,58.0
1,784,100,4.008440e+06,1481,1270,730,37,7542.67,No,NaN
2,785,200,3.983578e+06,1498,1287,762,37,7629.25,No,NaN
3,787,400,3.934566e+06,1506,1295,804,38,7670.00,No,NaN
4,788,500,3.911622e+06,1496,1283,812,38,7619.07,Yes,675.0
5,790,700,3.869047e+06,1454,1240,813,40,7405.16,No,NaN
6,791,800,3.797914e+06,1427,1212,806,42,7267.65,No,NaN
7,792,900,3.673538e+06,1396,1182,791,42,7109.77,No,NaN
8,793,1000,3.542189e+06,1357,1145,771,43,6911.14,No,NaN
9,794,1100,3.408509e+06,1314,1100,746,44,6692.15,No,NaN


In [26]:
transfer_distances = r25[r25['Transfer'] == 'Yes']['Distance from initial stop (feet)'].values
transfer_distances = transfer_distances.reshape((1, transfer_distances.shape[0]))
transfer_distances.shape

(1, 8)

In [28]:
non_transfer_distances = r25[r25['Transfer'] == 'No']['Distance from initial stop (feet)'].values
non_transfer_distances = non_transfer_distances.reshape((non_transfer_distances.shape[0], 1))
non_transfer_distances.shape

(562, 1)

In [30]:
transfer_matrix = np.repeat(transfer_distances, non_transfer_distances.shape[0], axis=0)
transfer_matrix.shape

(562, 8)

In [34]:
non_transfer_matrix = np.repeat(non_transfer_distances, transfer_distances.shape[1], axis=1)
non_transfer_matrix.shape

(562, 8)

In [36]:
distances_from_transfers = np.absolute(transfer_matrix - non_transfer_matrix)

In [38]:
fivehundred = 500 * np.ones(distances_from_transfers.shape)
less_than_500 = np.less(distances_from_transfers, fivehundred)

In [42]:
less_than_500

array([[ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True]])

In [51]:
stop_invalid = np.any(less_than_500, axis=1)
stop_invalid.shape

(562,)

In [52]:
stop_valid = np.logical_not(stop_invalid)
stop_valid.shape

(562,)

In [54]:
np.sum(stop_valid)

531

In [56]:
r25_valid = r25[r25['Transfer'] == 'No'][stop_valid]

In [57]:
r25_valid.head()

,UniqueNum,Distance from initial stop (feet),SqFtBuffere,Est_TotPop,Est_TotMinority,Est_TotPov,Est_TotLEP,Est_TotPop_Density,Transfer,CorrespondingStopID
17,818,3500,2.183919e+04,612,448,217,25,3116.89,No,1796.0
18,819,3600,3.728755e+06,597,438,210,24,3040.50,No,NaN
19,820,3700,3.923674e+06,582,427,203,22,2964.10,No,NaN
20,821,3800,4.108728e+06,568,419,199,21,2892.80,No,NaN
21,822,3900,4.278930e+06,556,411,195,20,2831.68,No,NaN


In [60]:
r25_valid.to_csv('../data/route_25_valid_nontransfer.csv', index=False)